In [1]:
import pandas as pd
import os
import numpy as np
import datetime

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Projection_est_in_Q3/JT_Rolling_Tracking'

In [2]:
def recurisve_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)

all_files_2019_daily=list(recurisve_file_gen("/home/jian/BigLots/2019_by_weeks/"))
all_files_2019_daily=[x for x in all_files_2019_daily if (".txt" in x) & ("aily" in x) ]
df_2019_daily=pd.DataFrame({"file_path":all_files_2019_daily})
df_2019_daily['week_end_dt']=df_2019_daily['file_path'].apply(lambda x: x.split("/2019_by_weeks/MediaStorm_")[1][:10])

df_2019_daily_Q3=df_2019_daily[(df_2019_daily['week_end_dt']>="2019-08-04")]
print("df_2019_daily_Q3['week_end_dt'].min(): "+str(df_2019_daily_Q3['week_end_dt'].min()))
print("df_2019_daily_Q3['week_end_dt'].max(): "+str(df_2019_daily_Q3['week_end_dt'].max()))
print("df_2019_daily_Q3.shape",df_2019_daily_Q3.shape)                        

df_2019_daily_Q3['week_end_dt'].min(): 2019-08-10
df_2019_daily_Q3['week_end_dt'].max(): 2019-09-07
df_2019_daily_Q3.shape (5, 2)


In [3]:
df_Q3_rewards_trans_store_2019=pd.DataFrame()
df_Q3_Nonrewards_trans_store_2019=pd.DataFrame()
for file in df_2019_daily_Q3['file_path'].tolist():
    df=pd.read_table(file,dtype=str,sep="|",
                     usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','item_transaction_amt'])
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    
    # Non-rewards
    df_non_rewards=df[pd.isnull(df['customer_id_hashed'])]
    df_non_rewards_sales=df_non_rewards.groupby(["location_id",'transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales"})
    df_non_rewards=df_non_rewards[['location_id','transaction_dt','transaction_id']].drop_duplicates()
    df_non_rewards=df_non_rewards.groupby(['location_id','transaction_dt'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    df_non_rewards=pd.merge(df_non_rewards_sales,df_non_rewards,on=['location_id','transaction_dt'],how="outer")
    df_Q3_Nonrewards_trans_store_2019=df_Q3_Nonrewards_trans_store_2019.append(df_non_rewards)
    
    # Rewards below
    df=df[pd.notnull(df['customer_id_hashed'])]
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans=df_trans.groupby(['customer_id_hashed','location_id','transaction_dt'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    df_sales=df.groupby(['customer_id_hashed','location_id','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales"})
    
    df=pd.merge(df_sales,df_trans,on=['customer_id_hashed','location_id','transaction_dt'],how="outer")
    df_Q3_rewards_trans_store_2019=df_Q3_rewards_trans_store_2019.append(df)
    
    

In [4]:
df_Q3_rewards_trans_store_2019['store_type']=np.where(df_Q3_rewards_trans_store_2019['location_id']=="6990","Online","Instore")
df_Q3_Nonrewards_trans_store_2019['store_type']=np.where(df_Q3_Nonrewards_trans_store_2019['location_id']=="6990","Online","Instore")


In [5]:
df_Q3_rewards_trans_type_2019=df_Q3_rewards_trans_store_2019.groupby(['customer_id_hashed','store_type'])['trans','sales'].sum().reset_index()
df_Q3_Nonrewards_trans_type_2019=df_Q3_Nonrewards_trans_store_2019.groupby(['store_type'])['trans','sales'].sum().reset_index()


In [6]:
df_Q3_rewards_trans_type_2019['both_type']="Both_Online_and_Instore"
df_Q3_Nonrewards_trans_type_2019['both_type']="Both_Online_and_Instore"


In [7]:
agg_func_type={"trans":"sum","sales":"sum","customer_id_hashed":"count"}
agg_func_both={"trans":"sum","sales":"sum","customer_id_hashed":"nunique"}

df_output_type_rewards_2019=df_Q3_rewards_trans_type_2019.groupby(['store_type'])['trans','sales','customer_id_hashed'].agg(agg_func_type).reset_index()
df_output_both_rewards_2019=df_Q3_rewards_trans_type_2019.groupby(['both_type'])['trans','sales','customer_id_hashed'].agg(agg_func_both).reset_index()

df_output_type_rewards_2019['avg_trans_per_reward_id']=df_output_type_rewards_2019['trans']/df_output_type_rewards_2019['customer_id_hashed']
df_output_type_rewards_2019['AOV']=df_output_type_rewards_2019['sales']/df_output_type_rewards_2019['trans']

df_output_both_rewards_2019['avg_trans_per_reward_id']=df_output_both_rewards_2019['trans']/df_output_both_rewards_2019['customer_id_hashed']
df_output_both_rewards_2019['AOV']=df_output_both_rewards_2019['sales']/df_output_both_rewards_2019['trans']


In [8]:
df_output_type_Nonrewards_2019=df_Q3_Nonrewards_trans_type_2019.groupby(['store_type'])['trans','sales'].sum().reset_index()
df_output_both_Nonrewards_2019=df_Q3_Nonrewards_trans_type_2019.groupby(['both_type'])['trans','sales'].sum().reset_index()

df_output_type_Nonrewards_2019['AOV']=df_output_type_Nonrewards_2019['sales']/df_output_type_Nonrewards_2019['trans']

df_output_both_Nonrewards_2019['AOV']=df_output_both_Nonrewards_2019['sales']/df_output_both_Nonrewards_2019['trans']


# 2018

In [9]:
available_weeks_18=pd.to_datetime(df_2019_daily_Q3['week_end_dt'].max()).date()-datetime.timedelta(days=52*7)
available_weeks_18=str(available_weeks_18).replace("-","")
available_weeks_18

'20180908'

In [10]:
all_files_2018_historical=list(recurisve_file_gen("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/"))
all_files_2018_historical=[x for x in all_files_2018_historical if (".txt" in x)]
all_2018_item_files=pd.DataFrame({"file_path":all_files_2018_historical})
all_2018_item_files['week_end_dt']=all_2018_item_files['file_path'].apply(lambda x: x.split("ed/MediaStormDailySalesHistory")[1][:8])

all_2018_item_files=all_2018_item_files[(all_2018_item_files['week_end_dt']>="20180805") &\
                               (all_2018_item_files['week_end_dt']<=available_weeks_18)]

print("all_2018_item_files['week_end_dt'].min(): "+str(all_2018_item_files['week_end_dt'].min()))
print("all_2018_item_files['week_end_dt'].max(): "+str(all_2018_item_files['week_end_dt'].max()))
print("all_2018_item_files.shape",all_2018_item_files.shape)                    

all_2018_item_files['week_end_dt'].min(): 20180811
all_2018_item_files['week_end_dt'].max(): 20180908
all_2018_item_files.shape (5, 2)


In [11]:
# First N (5) weeks only so far
df_Q3_rewards_trans_store_2018=pd.DataFrame()
df_Q3_Nonrewards_trans_store_2018=pd.DataFrame()

for file in all_2018_item_files['file_path'].tolist():
    df=pd.read_table(file,dtype=str,sep="|",
                     usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','item_transaction_amt'])
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    
    # Non-rewards
    df_non_rewards=df[pd.isnull(df['customer_id_hashed'])]
    df_non_rewards_sales=df_non_rewards.groupby(["location_id",'transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales"})
    df_non_rewards=df_non_rewards[['location_id','transaction_dt','transaction_id']].drop_duplicates()
    df_non_rewards=df_non_rewards.groupby(['location_id','transaction_dt'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    df_non_rewards=pd.merge(df_non_rewards_sales,df_non_rewards,on=['location_id','transaction_dt'],how="outer")
    df_Q3_Nonrewards_trans_store_2018=df_Q3_Nonrewards_trans_store_2018.append(df_non_rewards)
    
    # Rewards below
    df=df[pd.notnull(df['customer_id_hashed'])]
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans=df_trans.groupby(['customer_id_hashed','location_id','transaction_dt'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    df_sales=df.groupby(['customer_id_hashed','location_id','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales"})
    
    df=pd.merge(df_sales,df_trans,on=['customer_id_hashed','location_id','transaction_dt'],how="outer")
    df_Q3_rewards_trans_store_2018=df_Q3_rewards_trans_store_2018.append(df)
    


In [12]:
df_Q3_rewards_trans_store_2018['store_type']=np.where(df_Q3_rewards_trans_store_2018['location_id']=="6990","Online","Instore")
df_Q3_Nonrewards_trans_store_2018['store_type']=np.where(df_Q3_Nonrewards_trans_store_2018['location_id']=="6990","Online","Instore")


In [13]:
df_Q3_rewards_trans_type_2018=df_Q3_rewards_trans_store_2018.groupby(['customer_id_hashed','store_type'])['trans','sales'].sum().reset_index()
df_Q3_Nonrewards_trans_type_2018=df_Q3_Nonrewards_trans_store_2018.groupby(['store_type'])['trans','sales'].sum().reset_index()


In [14]:
df_Q3_rewards_trans_type_2018['both_type']="Both_Online_and_Instore"
df_Q3_Nonrewards_trans_type_2018['both_type']="Both_Online_and_Instore"


In [15]:
agg_func_type={"trans":"sum","sales":"sum","customer_id_hashed":"count"}
agg_func_both={"trans":"sum","sales":"sum","customer_id_hashed":"nunique"}

df_output_type_rewards_2018=df_Q3_rewards_trans_type_2018.groupby(['store_type'])['trans','sales','customer_id_hashed'].agg(agg_func_type).reset_index()
df_output_both_rewards_2018=df_Q3_rewards_trans_type_2018.groupby(['both_type'])['trans','sales','customer_id_hashed'].agg(agg_func_both).reset_index()

df_output_type_rewards_2018['avg_trans_per_reward_id']=df_output_type_rewards_2018['trans']/df_output_type_rewards_2018['customer_id_hashed']
df_output_type_rewards_2018['AOV']=df_output_type_rewards_2018['sales']/df_output_type_rewards_2018['trans']

df_output_both_rewards_2018['avg_trans_per_reward_id']=df_output_both_rewards_2018['trans']/df_output_both_rewards_2018['customer_id_hashed']
df_output_both_rewards_2018['AOV']=df_output_both_rewards_2018['sales']/df_output_both_rewards_2018['trans']


In [16]:
df_output_type_Nonrewards_2018=df_Q3_Nonrewards_trans_type_2018.groupby(['store_type'])['trans','sales'].sum().reset_index()
df_output_both_Nonrewards_2018=df_Q3_Nonrewards_trans_type_2018.groupby(['both_type'])['trans','sales'].sum().reset_index()

df_output_type_Nonrewards_2018['AOV']=df_Q3_Nonrewards_trans_type_2018['sales']/df_Q3_Nonrewards_trans_type_2018['trans']

df_output_both_Nonrewards_2018['AOV']=df_output_both_Nonrewards_2018['sales']/df_output_both_Nonrewards_2018['trans']


In [17]:
all_files_2018_daily_later=list(recurisve_file_gen("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/"))
all_files_2018_daily_later=[x for x in all_files_2018_daily_later if x[-4:]==".txt"]
df_2018_daily_Q3_later=pd.DataFrame({"file_path":all_files_2018_daily_later})
df_2018_daily_Q3_later['week_end_dt']=df_2018_daily_Q3_later['file_path'].apply(lambda x: x.split("/MediaStormDailySalesHistory")[1][:8])

df_2018_daily_Q3_later=df_2018_daily_Q3_later[(df_2018_daily_Q3_later['week_end_dt']>available_weeks_18) &\
                               (df_2018_daily_Q3_later['week_end_dt']<="20181103")]

print("df_2018_daily_Q3_later['week_end_dt'].min(): "+str(df_2018_daily_Q3_later['week_end_dt'].min()))
print("df_2018_daily_Q3_later['week_end_dt'].max(): "+str(df_2018_daily_Q3_later['week_end_dt'].max()))
print("df_2018_daily_Q3_later.shape: "+str(df_2018_daily_Q3_later.shape))
            

df_2018_daily_Q3_later['week_end_dt'].min(): 20180915
df_2018_daily_Q3_later['week_end_dt'].max(): 20181103
df_2018_daily_Q3_later.shape: (8, 2)


In [18]:
# The whole Q3 2018 with 13 weeks
for file in df_2018_daily_Q3_later['file_path'].tolist():
    df=pd.read_table(file,dtype=str,sep="|",
                     usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed','item_transaction_amt'])
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    
    # Non-rewards
    df_non_rewards=df[pd.isnull(df['customer_id_hashed'])]
    df_non_rewards_sales=df_non_rewards.groupby(["location_id",'transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales"})
    df_non_rewards=df_non_rewards[['location_id','transaction_dt','transaction_id']].drop_duplicates()
    df_non_rewards=df_non_rewards.groupby(['location_id','transaction_dt'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    df_non_rewards=pd.merge(df_non_rewards_sales,df_non_rewards,on=['location_id','transaction_dt'],how="outer")
    df_Q3_Nonrewards_trans_store_2018=df_Q3_Nonrewards_trans_store_2018.append(df_non_rewards)
    
    # Rewards below
    df=df[pd.notnull(df['customer_id_hashed'])]
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans=df_trans.groupby(['customer_id_hashed','location_id','transaction_dt'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})
    df_sales=df.groupby(['customer_id_hashed','location_id','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"sales"})
    
    df=pd.merge(df_sales,df_trans,on=['customer_id_hashed','location_id','transaction_dt'],how="outer")
    df_Q3_rewards_trans_store_2018=df_Q3_rewards_trans_store_2018.append(df)
    


In [19]:
df_Q3_rewards_trans_store_2018['store_type']=np.where(df_Q3_rewards_trans_store_2018['location_id']=="6990","Online","Instore")
df_Q3_Nonrewards_trans_store_2018['store_type']=np.where(df_Q3_Nonrewards_trans_store_2018['location_id']=="6990","Online","Instore")

df_Q3_rewards_trans_type_2018=df_Q3_rewards_trans_store_2018.groupby(['customer_id_hashed','store_type'])['trans','sales'].sum().reset_index()
df_Q3_Nonrewards_trans_type_2018=df_Q3_Nonrewards_trans_store_2018.groupby(['store_type'])['trans','sales'].sum().reset_index()

df_Q3_rewards_trans_type_2018['both_type']="Both_Online_and_Instore"
df_Q3_Nonrewards_trans_type_2018['both_type']="Both_Online_and_Instore"


In [20]:
agg_func_type={"trans":"sum","sales":"sum","customer_id_hashed":"count"}
agg_func_both={"trans":"sum","sales":"sum","customer_id_hashed":"nunique"}

df_output_type_rewards_2018_whole=df_Q3_rewards_trans_type_2018.groupby(['store_type'])['trans','sales','customer_id_hashed'].agg(agg_func_type).reset_index()
df_output_both_rewards_2018_whole=df_Q3_rewards_trans_type_2018.groupby(['both_type'])['trans','sales','customer_id_hashed'].agg(agg_func_both).reset_index()

df_output_type_rewards_2018_whole['avg_trans_per_reward_id']=df_output_type_rewards_2018_whole['trans']/df_output_type_rewards_2018_whole['customer_id_hashed']
df_output_type_rewards_2018_whole['AOV']=df_output_type_rewards_2018_whole['sales']/df_output_type_rewards_2018_whole['trans']

df_output_both_rewards_2018_whole['avg_trans_per_reward_id']=df_output_both_rewards_2018_whole['trans']/df_output_both_rewards_2018_whole['customer_id_hashed']
df_output_both_rewards_2018_whole['AOV']=df_output_both_rewards_2018_whole['sales']/df_output_both_rewards_2018_whole['trans']


In [21]:
df_output_type_Nonrewards_2018_whole=df_Q3_Nonrewards_trans_type_2018.groupby(['store_type'])['trans','sales'].sum().reset_index()
df_output_both_Nonrewards_2018_whole=df_Q3_Nonrewards_trans_type_2018.groupby(['both_type'])['trans','sales'].sum().reset_index()

df_output_type_Nonrewards_2018_whole['AOV']=df_output_type_Nonrewards_2018_whole['sales']/df_output_type_Nonrewards_2018_whole['trans']

df_output_both_Nonrewards_2018_whole['AOV']=df_output_both_Nonrewards_2018_whole['sales']/df_output_both_Nonrewards_2018_whole['trans']


In [22]:
df_rewards_2018_store=df_Q3_rewards_trans_store_2018.groupby(["location_id","transaction_dt","store_type"])['sales','trans'].sum().reset_index()
df_rewards_2019_store=df_Q3_rewards_trans_store_2019.groupby(["location_id","transaction_dt","store_type"])['sales','trans'].sum().reset_index()

df_Nonrewards_2018_store=df_Q3_Nonrewards_trans_store_2018.groupby(["location_id","transaction_dt","store_type"])['sales','trans'].sum().reset_index()
df_Nonrewards_2019_store=df_Q3_Nonrewards_trans_store_2019.groupby(["location_id","transaction_dt","store_type"])['sales','trans'].sum().reset_index()


In [24]:
writer=pd.ExcelWriter("./BL_Q3_overall_5_weeks_18_19_tracking_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

df_output_type_rewards_2019.to_excel(writer,"R_online_and_instore_19_5W",index=False)
df_output_both_rewards_2019.to_excel(writer,"R_overall_19_5W",index=False)
df_output_type_Nonrewards_2019.to_excel(writer,"N_online_and_instore_19_5W",index=False)
df_output_both_Nonrewards_2019.to_excel(writer,"N_overall_19_5W",index=False)


df_output_type_rewards_2018.to_excel(writer,"R_online_and_instore_18_5W",index=False)
df_output_both_rewards_2018.to_excel(writer,"R_overall_18_5W",index=False)
df_output_type_Nonrewards_2018.to_excel(writer,"N_online_and_instore_18_5W",index=False)
df_output_both_Nonrewards_2018.to_excel(writer,"N_overall_18_5W",index=False)


df_output_type_rewards_2018_whole.to_excel(writer,"R_online_and_instore_18_13W",index=False)
df_output_both_rewards_2018_whole.to_excel(writer,"R_overall_18_13W",index=False)
df_output_type_Nonrewards_2018_whole.to_excel(writer,"N_online_and_instore_18_13W",index=False)
df_output_both_Nonrewards_2018_whole.to_excel(writer,"N_overall_18_13W",index=False)

df_rewards_2018_store.to_excel(writer,"df_rewards_2018_store",index=False)
df_rewards_2019_store.to_excel(writer,"df_rewards_2019_store",index=False)

df_Nonrewards_2018_store.to_excel(writer,"df_Nonrewards_2018_store",index=False)
df_Nonrewards_2019_store.to_excel(writer,"df_Nonrewards_2019_store",index=False)

writer.save()